# Práctica 3: Map-Reduce con datasets reales

En esta práctica veremos usos de MapReduce sobre datasets reales y relativamente grandes.

En este y en los siguientes ejercicios utilizaremos dos ficheros de entrada que han sido descargados del National Bureau of Economic Research (NBER) de EEUU (<http://www.nber.org/patents/>).

En concreto, usaremos los ficheros `cite75_99.txt` y `apat63_99.txt`. Podéis descargar estos dos ficheros, junto con el fichero `country_codes.txt`, desde [aquí](https://nubeusc-my.sharepoint.com/:u:/g/personal/tf_pena_usc_es/EVmESx6Ux4BCjA4nl916iNkBbpUjuzjnwTqrP_2nvv31rQ?e=mtA6dx).
Es este otro [enlace](https://nubeusc-my.sharepoint.com/:u:/g/personal/tf_pena_usc_es/EY_mV-NUz1RDiJNi4Qb2vYgBsCdvRyzExsrqXxABpqcXLA?e=wbsYqq) podéis descargar una versión reducida de los datos para hacer pruebas rápidas.

Una descripción detallada de estos ficheros puede encontrarse en:

Hall, B. H., A. B. Jaffe, and M. Trajtenberg (2001). *The NBER Patent Citation Data File: Lessons, Insights and Methodological Tools*. NBER Working Paper 8498.

## Fichero `cite75_99.txt`

Este fichero contiene citas de patentes emitidas entre 1975 y 1990 en los EEUU. Es un fichero CSV (*comma-separated values*) con más de 16,5 millones de filas, y las primeras líneas son como sigue:

```
"CITING","CITED"
3858241,956203
3858241,1324234
3858241,3398406
3858241,3557384
3858241,3634889
3858242,1515701
3858242,3319261
3858242,3668705
..., ...
```

La primera línea contiene una cabecera con la descripción de las columnas. Cada una de las otras líneas indica una cita que la patente con el número de la primera columna ha hecho a la patente con el número en la segunda.  Por ejemplo, la segunda fila indica que la patente nº 3858241 ("citing" o *citante*) hace una cita a la patente nº 956203 ("cited" o citada). El fichero está ordenado por las patentes citantes. Así podemos ver que la patente nº 3858241 cita a otras 5 patentes.

Este fichero permite extraer conclusiones sobre las patentes que a primera vista están ocultas. Por ejemplo, en el siguiente gráfico\[1\] se muestra una vista parcial del grafo de citaciones entre patentes:

![](https://udemy-images.s3.amazonaws.com:443/redactor/raw/2017-07-10_16-55-34-cffc86d6343d9256d3607eb865c7fc35.png)

## Fichero `apat63_99.txt`

Este fichero contiene una descripción de las patentes. Es, de nuevo, un fichero CSV e incluye, entre otros campos, el número de patente, el año de solicitud ("APPYEAR"), el año de concesión ("GYEAR"), el país del primer inventor ("COUNTRY"), el número de reivindicaciones ("CLAIMS") y otros metadatos acerca de las patentes. Tiene más de 2,9 millones de filas, y las primeras de estas son:

```
"PATENT","GYEAR","GDATE","APPYEAR","COUNTRY","POSTATE","ASSIGNEE","ASSCODE","CLAIMS","NCLASS","CAT","SUBCAT","CMADE","CRECEIVE","RATIOCIT","GENERAL","ORIGINAL","FWDAPLAG","BCKGTLAG","SELFCTUB","SELFCTLB","SECDUPBD","SECDLWBD"
3070801,1963,1096,,"BE","",,1,,269,6,69,,1,,0,,,,,,,
3070802,1963,1096,,"US","TX",,1,,2,6,63,,0,,,,,,,,,
3070803,1963,1096,,"US","IL",,1,,2,6,63,,9,,0.3704,,,,,,,
3070804,1963,1096,,"US","OH",,1,,2,6,63,,3,,0.6667,,,,,,,
3070805,1963,1096,,"US","CA",,1,,2,6,63,,1,,0,,,,,,,
3070806,1963,1096,,"US","PA",,1,,2,6,63,,0,,,,,,,,,
3070807,1963,1096,,"US","OH",,1,,623,3,39,,3,,0.4444,,,,,,,
3070808,1963,1096,,"US","IA",,1,,623,3,39,,4,,0.375,,,,,,,
...
```

Al igual que en muchos datasets reales, algunos de los valores de este fichero están vacíos.


## Fichero `country_codes.txt`

Es un pequeño fichero conteniendo el nombre completo de los países
correspondientes a los códigos `COUNTRY` del fichero `apat63_99.txt`.

## **Actividad**: Programación en Hadoop

1. Plantilla **01-citingpatents**: programa MapReduce escrito en Java que, para cada patente de `cite75_99.txt`, obtenga la lista de las que la citan

   - Formato salida: `patente patente1,patente2...` (la separación entre la clave y los valores debe ser un tabulado)

     - El mapper debe obtener cada línea del fichero de entrada, separar los campos y darle la vuelta (para obtener como clave intermedia la patente citada y como valor intermedio la patente que la cita):
       - 3858245,3755824 → 3755824 3858245

     - El reducer, para cada patente recibe como valor una lista de las que la citan, y tiene que convertir esa lista en un string: 
       - 3755824 {3858245 3858247. . . } → 3755824 3858245,3858247...

   - IMPORTANTE:
     - La cabecera del fichero no debe aparecer en la salida.
     - La lista de números de patente en el campo valor de la salida debe estar separada por comas, y no debe de haber una coma al final.
     - Para compilar la práctica y generar el fichero .jar, podéis hacerlo desde Eclipse o usando maven: `mvn package` (el fichero `.jar` se crea en el directorio target)


2. Plantilla **02-citationnumberbypatent_chained**: programa MapReduce que usa ChainMapper y ChainReducer para concatenar trabajos MapReduce

   - Obtener el número de citas de una patente, combinando el programa anterior **01-citingpatents** con un mapper adicional (`CCMapper`) que, a partir de la salida del reducer del 01-citingpatents, para cada patente, cuente el número de patentes que la citan. 

   - **IMPORTANTE**: la salida debe de estar en texto plano con dos columnas (patente, nº de citas) por línea, **separadas por una coma** (sin espacios en blanco).

   - Para compilar, copiad el fichero `citingpatents-0.0.1-SNAPSHOT.jar` generado en la práctica 1 al directorio `src/resources` de esta práctica, y usad Eclipse o maven para generar el nuevo .jar.

   - Para ejecutar el código, haced lo siguiente:

    ```bash
    export HADOOP_CLASSPATH="./citingpatents-0.0.1-SNAPSHOT.jar"
    yarn jar target/citationnumberbypatent_chained-0.0.1-SNAPSHOT.jar -libjars $HADOOP_CLASSPATH path_a_cite75_99.txt_en_HDFS dir_salida_en_HDFS
    ```


3. Plantilla **03-creasequencefile**: hacer un programa MapReduce MapOnly (sin reducers) que lea el fichero apat63_99.txt, separe los campos y lo guarde **como un fichero  Sequence** (formato clave/valor) con:
    - clave: el país (Text), **reemplazando el código por el nombre completo usando el fichero country_codes.txt**.
    - valor: una cadena (Text) con la patente y el año separados por coma, **sin espacios en blanco**.
    - El carácter de separación entre clave y valor debe ser el por defecto (tabulado).
    - **IMPORTANTE**:
        - El fichero `apat63_99.txt` no se puede modificar, por lo que nuestro código tiene que ser capaz de manejar la primera línea del fichero que es diferente a las demás.

        - El fichero `country_codes.txt` **debe residir en el disco local (no en HDFS)**. Para hacer que sea accesible mediante localización de dependencias, indicadlo con la opción **-files** al lanzar la tarea:

        ```bash
        yarn jar target/creasequencefile-0.0.1-SNAPSHOT.jar -files path_a_country_codes.txt_en_local path_a_apat63_99.txt_en_HDFS dir_salida_en_HDFS        
         
        ``` 

        - Fijar el número de reducers a 0 (MapOnly job).

        - La salida serán varios ficheros `part-m-*`. Se puede ver el contenido de estos ficheros ejecutando:

        ```
        hsfs dfs -text part-m-*
        ```


4. Plantilla **04-sortsecundario**: partiendo de los ficheros de salida del programa **03-creasequencefile**, obtener un fichero de texto con la siguiente información:
    -   país    año -> no de patentes ese año
    -   Es decir, deberemos tener una línea para cada país y año. Por ejemplo:
            Spain      1963->19
            Spain      1965->49
            ...
    -   **IMPORTANTE**: La salida debe estar ordenada por países, y para cada país, los valores ordenados por año  
    -   La ordenación de los valores debe realizarse mediante un **sort secundario** (ver, por ejemplo,
        https://www.oreilly.com/library/view/data-algorithms/9781491906170/ch01.html,
        o
        http://blog.ditullio.fr/2015/12/28/hadoop-basics-secondary-sort-in-mapreduce/).
    
    - Se debe usar una clave compuesta [país, año], de forma que

        -   Las claves se comparan usando la clave compuesta \[país, año\] para garantizar el ordenamiento correcto (establecido usando `job.setSortComparatorClass`).
        -   Las claves se particionan por país, garantizando que los mismos países van al mismo reducer (establecido usando `job.setPartitionerClass`)
        -   Los valores se agrupan por país (establecido usando `job.setGroupingComparatorClass`)


## **Entrega**

-   Para cada código, enviar un fichero comprimido incluyendo todo el proyecto desarrollado, con todas las fuentes y preparado para compilar usando Eclipse o maven
-   Incluir también un README con instrucciones de compilación y ejecución de cada uno de los programas.

#### Footnotes:

\[1\]. Gráfico extraído de: Chuck Lam, *Hadoop in action*, Manning Publications Co., 2011.
